Para ajustar las coordenadas de tus puntos utilizando las distancias medidas y las coordenadas iniciales, \
puedes utilizar el método de mínimos cuadrados no lineales. Vamos a definir un modelo que represente las distancias \
entre los puntos y luego ajustar este modelo a las distancias medidas.

Aquí tienes un ejemplo de cómo puedes hacerlo con `scipy.optimize.least_squares`:

### Paso 1: Importar las Bibliotecas Necesarias

In [1]:
import numpy as np
from scipy.optimize import least_squares
import matplotlib.pyplot as plt

### Paso 2: Definir el Modelo

Definimos una función que calcule las distancias entre los puntos en función de sus coordenadas.

In [2]:
# Definir el modelo
def calcular_distancia(p1, p2):
    return np.linalg.norm(p1 - p2)


def residuals(params, puntos, observaciones, pesos):
    resid = []
    params = params.reshape(-1, 3)  # Reorganizar los parámetros en una matriz de coordenadas
    for i, obs in enumerate(observaciones):
        idx1 = np.where(puntos[:, 0] == obs[0])[0][0]
        idx2 = np.where(puntos[:, 0] == obs[1])[0][0]
        dist_calculada = calcular_distancia(params[idx1], params[idx2])
        dist_observada = float(obs[2])
        resid.append(pesos[i] * (dist_calculada - dist_observada))
    return resid


### Paso 3: Preparar los Datos

Organizamos las coordenadas iniciales y las observaciones.

In [3]:
puntos = np.array([
    ['10', [-922886.72715334, -5951569.11657866, 2099281.85726845]],
    ['11', [-923608.04171829, -5951534.04024824, 2099036.70315641]],
    ['4', [-924151.68281468, -5952372.84075829, 2096541.10163531]],
    ['3', [-924347.7783152, -5952308.43195145, 2096629.12382028]],
    ['8', [-921992.37525752, -5952223.15849636, 2097964.20772335]],
    ['9', [-921980.7985837, -5952077.42737261, 2098347.62710959]],
],dtype=object)

observaciones = np.array([
    ['10', '11', 762.566],  # Distancia medida entre 10 y 11
    ['4', '3', 224.384],    # Distancia medida entre 4 y 3
    ['11', '4', 2688.1995], # Distancia medida entre 11 y 4
    ['8', '4', 2590.4349],  # Distancia medida entre 8 y 4
    ['8', '9', 410.188],    # Distancia medida entre 8 y 9
    ['8', '11', 2057.4457],  # Distancia medida entre 8 y 11
], dtype=object)

# Convertir las coordenadas iniciales a un vector
coordenadas_iniciales = np.array([p[1] for p in puntos]).flatten()

### Paso 4: Realizar el Ajuste de Mínimos Cuadrados

In [4]:
# Calcular los pesos (inverso del cuadrado de la desviación estándar)
desviacion_estandar = .1  # metros
pesos = 1 / desviacion_estandar**2
pesos = np.full(observaciones.shape[0], pesos)

In [5]:
# Realizar el ajuste de mínimos cuadrados con los pesos
resultado = least_squares(residuals, coordenadas_iniciales, args=(puntos, observaciones, pesos), verbose=2)

# Reorganizar las coordenadas ajustadas en una matriz
coordenadas_ajustadas = resultado.x.reshape(-1, 3)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.9862e+03                                    5.08e+03    
       1             15         3.7400e+02      1.61e+03       2.33e-01       2.00e+03    
       2             16         2.9938e+02      7.46e+01       4.66e-01       1.19e+03    
       3             18         4.8264e+00      2.95e+02       2.33e-01       1.52e+02    
       4             20         4.8264e+00      0.00e+00       0.00e+00       1.52e+02    
`xtol` termination condition is satisfied.
Function evaluations 20, initial cost 1.9862e+03, final cost 4.8264e+00, first-order optimality 1.52e+02.


In [6]:
# Obtener la matriz de diseño (Jacobiana)
resultado.jac, resultado.active_mask

(array([[ 94.56941532,  -4.58980978,  32.18249342, -94.56960594,
           4.57820457, -32.17881634,  -0.        ,  -0.        ,
           0.        ,  -0.        ,  -0.        ,   0.        ,
          -0.        ,  -0.        ,   0.        ,  -0.        ,
          -0.        ,   0.        ],
        [ -0.        ,  -0.        ,   0.        ,  -0.        ,
          -0.        ,   0.        ,  87.37415648, -28.73055697,
         -39.25257043, -87.37560843,  28.6942876 ,  39.26434751,
          -0.        ,  -0.        ,   0.        ,  -0.        ,
          -0.        ,   0.        ],
        [ -0.        ,  -0.        ,   0.        ,  20.23328304,
          31.19734081,  92.8291564 , -20.2337742 , -31.20031894,
         -92.8289956 ,  -0.        ,  -0.        ,   0.        ,
          -0.        ,  -0.        ,   0.        ,  -0.        ,
          -0.        ,   0.        ],
        [ -0.        ,  -0.        ,   0.        ,  -0.        ,
          -0.        ,   0.        , -83.

### Paso 5: Mostrar los Resultados

In [40]:
print("Coordenadas ajustadas:")
for i, p in enumerate(puntos):
    print(f"Punto {p[0]}: {coordenadas_ajustadas[i]}")

Coordenadas ajustadas:
Punto 10: [ -922886.66263668 -5951569.11972655  2099281.8791796 ]
Punto 11: [ -923607.82778793 -5951534.16315106  2099036.47697081]
Punto 4: [ -924151.74581987 -5952372.85058309  2096541.04429403]
Punto 3: [ -924347.80576099 -5952308.42295753  2096629.13610006]
Punto 8: [ -921992.56511594 -5952223.05527656  2097964.39485403]
Punto 9: [ -921980.79672329 -5952077.40373227  2098347.68931868]


Este código ajusta las coordenadas de los puntos utilizando las distancias medidas y las coordenadas iniciales, \
minimizando la diferencia entre las distancias calculadas y las observadas. La función `least_squares` \
proporciona un ajuste robusto de mínimos cuadrados no lineales, y el resultado incluye las coordenadas ajustadas de cada punto.